<a href="https://colab.research.google.com/github/DAG00DAG33/Hackaton-DataCrunch/blob/main/DataCrunch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
import requests
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import cross_val_score


from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

path = "/gdrive/MyDrive/ML/DataCrunch/models/"

training_data = pd.read_pickle("/gdrive/MyDrive/ML/DataCrunch/data/train_test_hackathon.pkl")


# Targets to be predicted
targets = training_data[training_data.columns[-3:]]

# Using target as feature is a bad idea
for target in ['target_r', 'target_g', 'target_b']:
    training_data = training_data.drop(target, axis=1)

# Data for which you will submit your prediction
hackathon_data = pd.read_pickle("/gdrive/MyDrive/ML/DataCrunch/data/hackathon_data.pkl")

Mounted at /gdrive


In [ ]:
path = "/gdrive/MyDrive/ML/DataCrunch/"

In [ ]:
"""# Data Download (may take a few minutes depending on your network)
train_datalink = 'http://hackathon.datacrunch.com/data/train_test_hackathon.pkl'
hackathon_data_link = 'http://hackathon.datacrunch.com/data/hackathon_data.pkl'

# Data for training
training_data = pd.read_pickle(train_datalink)

# Targets to be predicted
targets = training_data[training_data.columns[-3:]]

# Using target as feature is a bad idea
for target in ['target_r', 'target_g', 'target_b']:
    training_data = training_data.drop(target, axis=1)

# Data for which you will submit your prediction
hackathon_data = pd.read_pickle(hackathon_data_link)"""

"# Data Download (may take a few minutes depending on your network)\ntrain_datalink = 'http://hackathon.datacrunch.com/data/train_test_hackathon.pkl'\nhackathon_data_link = 'http://hackathon.datacrunch.com/data/hackathon_data.pkl'\n\n# Data for training\ntraining_data = pd.read_pickle(train_datalink)\n\n# Targets to be predicted\ntargets = training_data[training_data.columns[-3:]]\n\n# Using target as feature is a bad idea\nfor target in ['target_r', 'target_g', 'target_b']:\n    training_data = training_data.drop(target, axis=1)\n\n# Data for which you will submit your prediction\nhackathon_data = pd.read_pickle(hackathon_data_link)"

In [ ]:
binary_atr = ['Feature_14_cv_bool', 'Feature_15_cv_bool', 'Feature_16_cv_bool', 'Feature_17_cv_bool', 'Feature_18_adv_cv_bool', 'Feature_19_adv_cv_bool', 'Feature_20_adv_cv_bool', 'Feature_21_adv_cv_bool', 'Feature_22_adv_cv_bool']
enum_atr = ['Feature_1_cv_enum', 'Feature_23_adv_cv_enum']
targets_atr = ['target_r', 'target_g', 'target_b']
nan_atr = ['Feature_7', 'Feature_8', 'Feature_9', 'Feature_11']
cols_to_norm = ["Feature_2", "Feature_3", "Feature_4", "Feature_6", "Feature_7", "Feature_11"]
atributes = [atr for atr in training_data.columns if atr not in targets_atr]
numeric_atr = [atr for atr in atributes if atr not in (binary_atr + enum_atr)]


In [ ]:
training_data.describe()

,Feature_1_cv_enum,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14_cv_bool,Feature_15_cv_bool,Feature_16_cv_bool,Feature_17_cv_bool,Feature_18_adv_cv_bool,Feature_19_adv_cv_bool,Feature_20_adv_cv_bool,Feature_21_adv_cv_bool,Feature_22_adv_cv_bool,Feature_23_adv_cv_enum
count,300000.000000,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,2.046110e+05,214564.000000,204506.000000,3.000000e+05,2.610880e+05,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,2.367430,3.768100e+05,3.093938e+06,3.704056e+05,3.601329e+03,3.776893e+05,1.490732e+04,2519.349161,-0.020086,4.908759e+10,8.380347e+09,inf,0.665203,0.490037,0.410513,0.023157,0.076293,0.084870,0.022287,0.133607,0.198230,0.623147,3.201337
std,0.723846,3.832078e+07,2.439559e+08,3.847110e+07,8.185623e+05,3.832163e+07,2.762918e+06,308.948786,1.608483,1.254558e+13,2.731617e+12,NaN,0.471920,0.499902,0.491928,0.150401,0.265467,0.278689,0.147615,0.340230,0.398667,0.484598,1.247063
min,0.000000,1.000000e+00,0.000000e+00,-1.008232e+09,0.000000e+00,1.000000e+00,-3.433611e+00,1851.859985,-6.495034,0.000000e+00,-2.060307e+07,7.331550e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,8.090000e+02,1.395000e+04,7.340000e+02,0.000000e+00,8.140000e+02,1.318669e+01,2274.639893,-0.281288,0.000000e+00,3.189688e+05,1.502053e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,2.000000,3.204000e+03,4.147417e+04,3.000000e+03,0.000000e+00,3.233000e+03,2.766816e+01,2601.419922,-0.084317,0.000000e+00,1.015865e+06,8.967701e-02,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
75%,3.000000,1.250000e+04,1.575090e+05,1.150800e+04,1.558000e+01,1.265800e+04,5.547611e+01,2779.030029,0.128861,1.999371e+04,3.572125e+06,2.722777e-01,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
max,3.000000,1.310950e+10,9.222350e+10,1.310950e+10,3.000000e+08,1.310950e+10,6.822500e+08,3025.860107,448.924598,4.504504e+15,1.266173e+15,inf,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion


from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, atr_names):
    self.atr_names = atr_names
  def fit(self, X, y = None):
    return self
  def transform(self, data):
    #print("dataFrameSel....")
    return data[self.atr_names]

class Log_data(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  def fit(self, X, y = None):
    return self
  def transform(self, data):
    #print("log...")
    return np.log(np.asarray(data))

class RemoveInf(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  def fit(self, X, y = None):
    return self
  def transform(self, data):
    #print("inf...")
    return pd.DataFrame(data).replace([np.inf, -np.inf], np.nan)

class ExtraFeature(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  def fit(self, X, y = None):
    return self
  def transform(self, data):
    data["Feature_nan"] = pd.isna(data["Feature_7"]).astype(int)
    #print("aaa")
    #print(data.columns)
    return data.values

"""class Add_nan(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  def fit(self, X, y = None):
    return self
  def transform(self, data):
    #print("inf...")
    return pd.DataFrame(data).replace([np.inf, -np.inf], np.nan)"""

enum_pipeline = Pipeline([
                ('selector', DataFrameSelector(enum_atr)),
                ('inputer', SimpleImputer(strategy="most_frequent")),
                ('oneHotEncoder', OneHotEncoder()),
])

num_pipeline = Pipeline([
                ('selector', DataFrameSelector(numeric_atr)),
                ('remove_inf', RemoveInf()),
                ('include_nan', ExtraFeature()),
                ('inputer', SimpleImputer(strategy="median")),
                ('log', Log_data()),
                ('remove_inf2', RemoveInf()),
                ('std_scaler', StandardScaler()),
                ('inputer2', SimpleImputer(strategy="constant", fill_value = 10)),
])

bin_pipeline = Pipeline([
                ('selector', DataFrameSelector(binary_atr)),
                ('inputer', SimpleImputer(strategy="most_frequent")),
])

full_pipeline = FeatureUnion(transformer_list = [
                ('enum_pipeline', enum_pipeline),
                ('num_pipeline', num_pipeline),
                ('bin_pipeline', bin_pipeline),
])

In [ ]:
X = pd.DataFrame(full_pipeline.fit_transform(training_data).toarray())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in log


In [ ]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.00000,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,0.023157,0.076293,0.410513,0.490037,0.084870,0.02076,0.125680,0.145543,0.623147,2.735153e-16,0.029567,0.138000,5.546767,-9.979504e-16,0.001967,1.194786e-14,7.358000,5.546767,0.011333,2.301639e-16,3.347967,6.820367,0.490037,0.410513,0.023157,0.076293,0.084870,0.022287,0.133607,0.198230,0.623147
std,0.150401,0.265467,0.491928,0.499902,0.278689,0.14258,0.331489,0.352648,0.484598,1.000002e+00,1.136591,1.532046,5.014624,1.000002e+00,1.009688,1.000002e+00,4.438930,5.014624,1.054550,1.000002e+00,4.719201,4.656859,0.499902,0.491928,0.150401,0.265467,0.278689,0.147615,0.340230,0.398667,0.484598
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-3.292658e+00,-4.896712,-3.278524,-9.336277,-3.293090e+00,-13.858513,-2.847422e+00,-6.859854,-6.006601,-11.186259,-5.917666e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-5.323504e-01,-0.611423,-0.518399,0.274328,-5.321209e-01,-0.254233,-5.093937e-01,1.397395,0.111988,-0.549330,-5.539180e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,3.504520e-02,-0.123062,0.044777,10.000000,3.606251e-02,0.085323,2.615084e-01,10.000000,10.000000,-0.068417,2.266096e-01,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,1.000000,5.962456e-01,0.479308,0.613408,10.000000,5.983439e-01,0.422255,6.775223e-01,10.000000,10.000000,0.451488,7.107365e-01,10.000000,10.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,6.311242e+00,10.000000,10.000000,10.000000,6.304280e+00,16.085422,1.644042e+00,10.000000,10.506698,10.543031,8.474949e+00,10.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
y = targets

#20% validation 20% test  60% train
#60000          60000     180000

X_d, X_val, y_d, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_d, y_d, test_size=0.25, random_state=42)

"""X_pca_d, X_val_pca, y_d, y_val = train_test_split(X_pca, y, test_size=0.2, random_state=42)
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca_d, y_d, test_size=0.25, random_state=42)

X_drop_d, X_val_drop, y_d, y_val = train_test_split(X_drop, y, test_size=0.2, random_state=42)
X_train_drop, X_test_drop, y_train, y_test = train_test_split(X_drop_d, y_d, test_size=0.25, random_state=42)"""


'X_pca_d, X_val_pca, y_d, y_val = train_test_split(X_pca, y, test_size=0.2, random_state=42)\nX_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca_d, y_d, test_size=0.25, random_state=42)\n\nX_drop_d, X_val_drop, y_d, y_val = train_test_split(X_drop, y, test_size=0.2, random_state=42)\nX_train_drop, X_test_drop, y_train, y_test = train_test_split(X_drop_d, y_d, test_size=0.25, random_state=42)'

In [ ]:
X_train.shape

(180000, 31)

In [ ]:
def bin(n, s):
  arr = np.zeros((s))
  for i in range(s - 1, -1, -1):
    if n >= pow(2, i):
      arr[s - i - 1] = 1
      n -= pow(2, i)
      #print(pow(2, i), i)
  return arr

"""
Compara las predicciones de n modelos y las reales para ver como de correlacionados son los resultados
luego he descubierto que era mas facil con spearman
"""
def compare(preds, y):
  preds = np.asarray(preds).transpose()
  y = np.asarray(y)
  arr = np.concatenate((preds, y), axis=1)
  di = [0] * pow(2, arr.shape[1]);
  #for i in range(pow(2, arr.shape[1])):
   # di[i] = 0
  for i in range(pow(2, arr.shape[1])):
    for j in range(arr.shape[0]):
      if np.all(arr[j] == bin(i, arr.shape[1])):
        di[i] = di[i] + 1
        #print(i, j)
  for i in range(pow(2, arr.shape[1])):
    print(bin(i, arr.shape[1]), "\t",  di[i]/arr.shape[0])
  print()


In [ ]:
def compareModel(model1, model2):
  X_pred1 = model1.predict(X_test)
  X_pred2 = model2.predict(X_test)

  compare([X_pred1[:, 0], X_pred2[:, 0]], y_test.values[:, 0].reshape(-1, 1))
  compare([X_pred1[:, 1], X_pred2[:, 1]], y_test.values[:, 1].reshape(-1, 1))
  compare([X_pred1[:, 2], X_pred2[:, 2]], y_test.values[:, 2].reshape(-1, 1))

In [ ]:
bin(63,10)

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1.])

In [ ]:
def accuracy(model, x, y):
  y = y.reshape(x.shape[0], -1)
  #print(y.shape)
  y_pred = model.predict(x).reshape(x.shape[0], -1)
  y_pred_p = (y_pred > 0.5).astype(int)

  for j in range(y_pred_p.shape[1]):
    good = 0
    tot = 0
    for i in range(y_pred_p.shape[0]):
      if y_pred_p[i][j] == y[i][j]:
        good += 1
      tot += 1
    print(good/tot)
    #print(y_pred)
  print()


#Table of models
| |train|test|val|
|-|-----|----|---|
|forest||(83'82 84'58 85'34)|(84'09 84'57 85'55)|
|forest_drop||(82'29 83'11 84'15)|(82'64 83'19 84'11)|
|forest_pca||(73'50 74'71 76'25)|(73'82 74'31 76'39)|
|forest_join||(79'97 80'79 81'89)|(80'17 80'77 81'80)|
|||||
|extra||(73'24 74'02 75'65)|(72'93 73'90 75'88)|
|||||
|ada1||(78'89 79'37 80'78)|(78'96 79'75 80'78)|
|ada2||||
|||||
|neigh||(66'77 67'63 69'33)|(66'77 67'67 69'18)|
|||||


# NN

In [ ]:
from keras.layers import Input, Dense, Dropout, BatchNormalization  
from keras.models import Model

def model(len):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(len)

    X = BatchNormalization()(X_input)
    X = Dropout(0.1)(X)
    X = Dense(30, activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Dropout(0.3)(X)
    X = Dense(30, activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Dropout(0.3)(X)
    X = Dense(10, activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Dense(10, activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Dense(10, activation = "relu")(X)
    X = BatchNormalization()(X)
    X = Dense(3, activation='sigmoid')(X)


    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='test_model')

    return model

test_model = model(X2.shape[1])
test_model.compile(loss='BinaryCrossentropy') #BinaryCrossentropy


In [ ]:
history  = test_model.fit(X2, np.array(y_train), epochs = 20, batch_size = 1000)
plt.plot(history.history['loss'])

In [ ]:
test_model.summary()

#Decision tree

In [ ]:

from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train.values[:, 0])
y_pred = tree_clf.predict(X_test)

In [ ]:
accuracy(tree_clf, X_test, y_test.values[:, 0])

0.7746166666666666



In [ ]:
compare([y_pred, y_pred], y_test.values[:, 0].reshape(-1, 1))

[0. 0. 0.] 	 0.4629666666666667
[0. 0. 1.] 	 0.1128
[0. 1. 0.] 	 0.0
[0. 1. 1.] 	 0.0
[1. 0. 0.] 	 0.0
[1. 0. 1.] 	 0.0
[1. 1. 0.] 	 0.11258333333333333
[1. 1. 1.] 	 0.31165



#Random Forest

##Normal

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time

class ForestSep(BaseEstimator, ClassifierMixin): #uno para cada parametro a predecir. Luego vi que no mejora a un solo modelo para los 3
  def __init__(self, n_estimators = 1000):
    self.forest_1 = RandomForestClassifier(bootstrap=False, max_features=20, n_estimators = n_estimators)
    self.forest_2 = RandomForestClassifier(bootstrap=False, max_features=20, n_estimators = n_estimators)
    self.forest_3 = RandomForestClassifier(bootstrap=False, max_features=20, n_estimators = n_estimators)
  def fit(self, X, y):
    y = np.asarray(y)
    self.forest_1.fit(X, y[:, 0])
    print("trained 1")
    self.forest_2.fit(X, y[:, 1])
    print("trained 2")
    self.forest_3.fit(X, y[:, 2])
    print("trained 3")
  def predict(self, X):
    pred_1 = self.forest_1.predict(X).reshape(-1, 1)
    pred_2 = self.forest_2.predict(X).reshape(-1, 1)
    pred_3 = self.forest_3.predict(X).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)
  def predict_proba(self, x):
    pred_1 = self.forest_1.predict_proba(X).reshape(-1, 1)
    pred_2 = self.forest_2.predict_proba(X).reshape(-1, 1)
    pred_3 = self.forest_3.predict_proba(X).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)

#forest_sep = ForestSep(n_estimators = 10)
forest_tog = RandomForestClassifier(bootstrap=False, max_features=20, n_estimators= 300)

t = time.time()
#forest_sep.fit(X_train, y_train.values)
#forest_tog.fit(X_train, y_train)
print(time.time() - t)



957.5377616882324


In [ ]:
accuracy(forest_tog, X_val, y_val.values)
#accuracy(forest_sep, X_test, y_test.values)

0.8408833333333333
0.8457333333333333
0.8555333333333334



In [ ]:
X_pred_tog = forest_tog.predict(X_test)
X_pred_sep = forest_sep.predict(X_test)

In [ ]:
compare([X_pred_sep[:, 0], X_pred_tog[:, 0]], y_test.values[:, 0].reshape(-1, 1))
compare([X_pred_sep[:, 1], X_pred_tog[:, 1]], y_test.values[:, 1].reshape(-1, 1))
compare([X_pred_sep[:, 2], X_pred_tog[:, 2]], y_test.values[:, 2].reshape(-1, 1))

[0. 0. 0.] 	 0.39413333333333334
[0. 0. 1.] 	 0.05446666666666666
[0. 1. 0.] 	 0.06145
[0. 1. 1.] 	 0.06456666666666666
[1. 0. 0.] 	 0.06663333333333334
[1. 0. 1.] 	 0.061516666666666664
[1. 1. 0.] 	 0.05333333333333334
[1. 1. 1.] 	 0.2439

[0. 0. 0.] 	 0.4023833333333333
[0. 0. 1.] 	 0.05098333333333333
[0. 1. 0.] 	 0.06055
[0. 1. 1.] 	 0.06441666666666666
[1. 0. 0.] 	 0.06695
[1. 0. 1.] 	 0.059283333333333334
[1. 1. 0.] 	 0.0519
[1. 1. 1.] 	 0.24353333333333332

[0. 0. 0.] 	 0.41385
[0. 0. 1.] 	 0.053366666666666666
[0. 1. 0.] 	 0.057666666666666665
[0. 1. 1.] 	 0.07663333333333333
[1. 0. 0.] 	 0.08095
[1. 0. 1.] 	 0.054683333333333334
[1. 1. 0.] 	 0.049183333333333336
[1. 1. 1.] 	 0.21366666666666667



In [ ]:
from sklearn.externals import joblib

joblib.dump(forest_tog, path + "forest_tog300.pk")

['/gdrive/MyDrive/ML/DataCrunch/models/forest_tog300.pk']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.externals import joblib

forest_tog = joblib.load(path + "forest_tog300.pk")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
for name, score in zip(X.columns, forest_tog.feature_importances_):#Freature selection
  print(score)

0.00047133919416389707
0.0013503435086816952
0.004213750042996247
0.003771890153568458
0.0028140695223395256
0.0011280889718911938
0.003055181453157459
0.0033382767228269878
0.0034550989205092194
0.05901358296693257
0.05924317386722408
0.06585041182849721
0.06906154494850429
0.06135801995527331
0.1471413958066277
0.13096915001677092
0.05815519173213166
0.03900565553582767
0.16036408577093336
0.06333514185125165
0.019219063697785026
0.016578479520402922
0.004425019395575224
0.004222144514471291
0.0011229245412461045
0.0014459268525725726
0.002565378662626692
0.0011995391008096583
0.0030187115077911983
0.005100982209359026
0.004006437227251175


##Drop

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time

"""Drop las features que no son importantes. Deberia generalizar mejor"""
class ForestSep_drop(BaseEstimator, ClassifierMixin):#Que sean tres modelos SEParados no mejora nada
  def __init__(self, bootstrap = False, n_estimators = 1000, max_features = "auto"):
    self.forest_1 = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators)
    self.forest_2 = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators)
    self.forest_3 = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators)
  def fit(self, X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    self.forest_1.fit(X[:, 9:19], y[:, 0])
    print("trained 1")
    self.forest_2.fit(X[:, 9:19], y[:, 1])
    print("trained 2")
    self.forest_3.fit(X[:, 9:19], y[:, 2])
    print("trained 3")
    return self
  def predict(self, X):
    X = np.asarray(X)
    x = X[:, 9:19]
    pred_1 = self.forest_1.predict(x).reshape(-1, 1)
    pred_2 = self.forest_2.predict(x).reshape(-1, 1)
    pred_3 = self.forest_3.predict(x).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)
  def predict_proba(self, X):
    X = np.asarray(X)
    x = X[:, 9:19]
    pred_1 = self.forest_1.predict_proba(X).reshape(-1, 1)
    pred_2 = self.forest_2.predict_proba(X).reshape(-1, 1)
    pred_3 = self.forest_3.predict_proba(X).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)
  def set_params(self, **parameters):#necesario para el hyperparameter tunning
    self.forest_1.set_params(**parameters)
    self.forest_2.set_params(**parameters)
    self.forest_3.set_params(**parameters)
    return self
  def get_params(self, deep=True):
    return {"bootstrap": self.forest_1.get_params()['bootstrap'], 
             "n_estimators": self.forest_1.get_params()['n_estimators'],
              "max_features": self.forest_1.get_params()['max_features']
              }

#forest_sep_drop = ForestSep_drop(n_estimators = 300, max_features = 6)

t = time.time()
#forest_sep_drop.fit(X_train, y_train.values)
print(time.time() - t)

2.9087066650390625e-05


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time

"""Drop las features que no son importantes. Deberia generalizar mejor"""
class ForestTog_drop(BaseEstimator, ClassifierMixin):
  def __init__(self, bootstrap = False, n_estimators = 1000, max_features = 6):
    self.forest = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators)
  def fit(self, X, y):
    X = np.asarray(X)
    x = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    #x = X[:, 9:19]
    #print(x.shape)
    y = np.asarray(y)
    self.forest.fit(x, y)
    return self
  def predict(self, X):
    X = np.asarray(X)
    x = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    #x = X[:, 9:19]
    return self.forest.predict(x)
  def predict_proba(self, X):
    X = np.asarray(X)
    x = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    #x = X[:, 9:19]
    return self.forest.predict_proba(x)
  def set_params(self, **parameters):
    self.forest.set_params(**parameters)
    return self
  def get_params(self, deep=True):
    return {"bootstrap": self.forest.get_params()['bootstrap'], 
             "n_estimators": self.forest.get_params()['n_estimators'],
              "max_features": self.forest.get_params()['max_features']
              }

#forest_tog_drop = ForestTog_drop(n_estimators = 300, max_features = 6)

t = time.time()
#forest_tog_drop.fit(X_train, y_train.values)
print(time.time() - t)

2.5033950805664062e-05


In [ ]:
accuracy(forest_tog_drop, X_test, y_test.values)
#accuracy(forest_sep_drop, X_test, y_test.values)

0.8228666666666666
0.83105
0.8415



In [ ]:
"""Comparar sep y tog"""
X_pred_tog_drop = forest_tog_drop.predict(X_test)
X_pred_sep_drop = forest_sep_drop.predict(X_test)

compare([X_pred_tog_drop[:, 0], X_pred_sep_drop[:, 0]], y_test.values[:, 0].reshape(-1, 1))
compare([X_pred_tog_drop[:, 1], X_pred_sep_drop[:, 1]], y_test.values[:, 1].reshape(-1, 1))
compare([X_pred_tog_drop[:, 2], X_pred_sep_drop[:, 2]], y_test.values[:, 2].reshape(-1, 1))

[0. 0. 0.] 	 0.46776666666666666
[0. 0. 1.] 	 0.0651
[0. 1. 0.] 	 0.0258
[0. 1. 1.] 	 0.024166666666666666
[1. 0. 0.] 	 0.023183333333333334
[1. 0. 1.] 	 0.03055
[1. 1. 0.] 	 0.0588
[1. 1. 1.] 	 0.3046333333333333

[0. 0. 0.] 	 0.47886666666666666
[0. 0. 1.] 	 0.06451666666666667
[0. 1. 0.] 	 0.025
[0. 1. 1.] 	 0.021466666666666665
[1. 0. 0.] 	 0.020683333333333335
[1. 0. 1.] 	 0.029383333333333334
[1. 1. 0.] 	 0.05723333333333333
[1. 1. 1.] 	 0.30285

[0. 0. 0.] 	 0.5081
[0. 0. 1.] 	 0.07068333333333333
[0. 1. 0.] 	 0.020216666666666668
[0. 1. 1.] 	 0.023983333333333332
[1. 0. 0.] 	 0.019
[1. 0. 1.] 	 0.021433333333333332
[1. 1. 0.] 	 0.05433333333333333
[1. 1. 1.] 	 0.28225



In [ ]:
from sklearn.externals import joblib

joblib.dump(forest_tog_drop, path + "forest_tog_drop300.pk")

['/gdrive/MyDrive/ML/DataCrunch/models/forest_tog_drop300.pk']

In [ ]:
from sklearn.externals import joblib

forest_tog_drop = joblib.load(path + "forest_tog_drop300.pk")

##PCA

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.decomposition import PCA

"""Drop de las features no importantes y PCA. El n_componenents es un hiperparametro"""
class ForestTog_pca(BaseEstimator, ClassifierMixin):
  def __init__(self, bootstrap = False, n_estimators = 10, max_features = 6, n_components = 11, max_depth = None):
    self.forest = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators, max_depth = max_depth)
    self.pca = PCA(n_components = n_components)
    #print("created")
  def fit(self, X, y):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    #print("start pca")
    X = self.pca.fit_transform(X)
    #print("pca done")
    y = np.asarray(y)
    self.forest.fit(X, y)
    #print("forest trained")
    print("trained")
    return self
  def predict(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    X = self.pca.transform(X)
    return self.forest.predict(X)
  def predict_proba(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    X = self.pca.transform(X)
    return self.forest.predict_proba(X)
  def set_params(self, **parameters):
    self.forest.set_params(**parameters)
    #self.pca.set_params(**parameters)
    return self
  def get_params(self, deep=True):
    return {"bootstrap": self.forest.get_params()['bootstrap'], 
            "n_estimators": self.forest.get_params()['n_estimators'],
            "max_features": self.forest.get_params()['max_features'],
            "n_components": self.pca.get_params()["n_components"]
            }

forest_tog_pca = ForestTog_pca(n_estimators = 100, max_features = 6, n_components = 11)

t = time.time()
#forest_tog_pca.fit(X_train, y_train.values)
print(time.time() - t)

2.4557113647460938e-05


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.decomposition import PCA

"""Igual, pero concatena el PCA y la features normales, parece no mejorar nada"""
class ForestTog_pcap(BaseEstimator, ClassifierMixin):
  def __init__(self, bootstrap = False, n_estimators = 10, max_features = 6, n_components = 11, max_depth = None):
    self.forest = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, n_estimators = n_estimators, max_depth = max_depth)
    self.pca = PCA(n_components = n_components)
  def fit(self, X, y):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    X = np.concatenate((X,self.pca.fit_transform(X)), axis = 1)
    y = np.asarray(y)
    self.forest.fit(X, y)
    print("trained")
    return self
  def predict(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    X = np.concatenate((X,self.pca.transform(X)), axis = 1)
    return self.forest.predict(X)
  def predict_proba(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    X = np.concatenate((X,self.pca.transform(X)), axis = 1)
    return self.forest.predict_proba(X)
  def set_params(self, **parameters):
    self.forest.set_params(**parameters)
    #self.pca.set_params(**parameters)
    return self
  def get_params(self, deep=True):
    return {"bootstrap": self.forest.get_params()['bootstrap'], 
            "n_estimators": self.forest.get_params()['n_estimators'],
            "max_features": self.forest.get_params()['max_features'],
            "n_components": self.pca.get_params()["n_components"]
            }

forest_tog_pcap = ForestTog_pcap(n_estimators = 100, max_features = 20, n_components = 11)

t = time.time()
#forest_tog_pcap.fit(X_train, y_train.values)
print(time.time() - t)

3.790855407714844e-05


In [ ]:
accuracy(forest_tog_pcap, X_test, y_test.values)

0.7727333333333334
0.7812
0.7949166666666667



In [ ]:
X_pred_tog_pcab = forest_tog_pca.predict(X_test)
X_pred_sep_pca = S_forest_tog_pca300.predict(X_test)

compare([X_pred_tog_pcab[:, 0], X_pred_sep_pca[:, 0]], y_test.values[:, 0].reshape(-1, 1))
compare([X_pred_tog_pcab[:, 1], X_pred_sep_pca[:, 1]], y_test.values[:, 1].reshape(-1, 1))
compare([X_pred_tog_pcab[:, 2], X_pred_sep_pca[:, 2]], y_test.values[:, 2].reshape(-1, 1))

[0. 0. 0.] 	 0.44503333333333334
[0. 0. 1.] 	 0.13383333333333333
[0. 1. 0.] 	 0.011416666666666667
[0. 1. 1.] 	 0.010983333333333333
[1. 0. 0.] 	 0.01055
[1. 0. 1.] 	 0.011233333333333333
[1. 1. 0.] 	 0.10855
[1. 1. 1.] 	 0.2684

[0. 0. 0.] 	 0.46058333333333334
[0. 0. 1.] 	 0.13068333333333335
[0. 1. 0.] 	 0.01025
[0. 1. 1.] 	 0.010416666666666666
[1. 0. 0.] 	 0.00995
[1. 0. 1.] 	 0.010933333333333333
[1. 1. 0.] 	 0.101
[1. 1. 1.] 	 0.2661833333333333

[0. 0. 0.] 	 0.4975333333333333
[0. 0. 1.] 	 0.13308333333333333
[0. 1. 0.] 	 0.009366666666666667
[0. 1. 1.] 	 0.009683333333333334
[1. 0. 0.] 	 0.0093
[1. 0. 1.] 	 0.0096
[1. 1. 0.] 	 0.08545
[1. 1. 1.] 	 0.24598333333333333



In [ ]:
for name, score in zip(X.columns, forest_tog_pcap.forest.feature_importances_):
  print(score)

0.027203381767329293
0.037542238839031634
0.03058797508406875
0.03288268162441428
0.027509721110265076
0.07670309623139847
0.08712393056786084
0.024075400492603878
0.015666435388998624
0.1260945880862815
0.0013076499196809637
0.04269325319475436
0.05148517115332211
0.031172652594143566
0.03784576756443117
0.04104699738131596
0.06124677701283131
0.04180570289880477
0.05440194320656051
0.05671548224140764
0.04608728226104742
0.04880187137944784


In [ ]:
from sklearn.externals import joblib

joblib.dump(forest_tog_pcap, path + "models/" + "forest_join300.pk")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['/gdrive/MyDrive/ML/DataCrunch/models/forest_join300.pk']

In [ ]:
from sklearn.externals import joblib

forest_tog_pca = joblib.load(path + "models/" + "forest_tog_pca300.pk")

##hyperparameter tuning

In [ ]:
forest_tog_pcap = ForestTog_pcap(n_estimators = 10, max_features = 16, n_components = 11)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'n_estimators': [10], "max_features": [10, 11, 12, 13, 14, 15, 16, 17]}
]

grid_search = GridSearchCV(forest_tog_pcap, param_grid, cv = 3, scoring = 'accuracy', refit = False)#ir cambiando el modelo

grid_search.fit(X_d, y_d.values)

In [ ]:
cvres = grid_search.cv_results_
for score, std, time, param in zip(cvres["mean_test_score"],cvres["std_test_score"] ,cvres["mean_fit_time"], cvres["params"]):
  print(param, "\t", score, "\t", std ,"\t", time)

{'max_features': 10, 'n_estimators': 10} 	 0.5902333333333334 	 0.009450718300155259 	 46.69732133547465
{'max_features': 11, 'n_estimators': 10} 	 0.5890208333333334 	 0.005440361635242804 	 50.902156035105385
{'max_features': 12, 'n_estimators': 10} 	 0.5927916666666667 	 0.004166662499997938 	 54.06312823295593
{'max_features': 13, 'n_estimators': 10} 	 0.5903125 	 0.002630767473571188 	 59.18082666397095
{'max_features': 14, 'n_estimators': 10} 	 0.5909833333333333 	 0.005112131540648066 	 62.02290280659994
{'max_features': 15, 'n_estimators': 10} 	 0.5907458333333334 	 0.0025175288258308037 	 66.31953692436218
{'max_features': 16, 'n_estimators': 10} 	 0.5934 	 0.006468384651518472 	 69.29738473892212
{'max_features': 17, 'n_estimators': 10} 	 0.590825 	 0.0011844038584874388 	 74.35504841804504


In [ ]:
grid_search.best_estimator_

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


ForestSep_drop(bootstrap=None, max_features=None, n_estimators=None)

In [ ]:
forest=RandomForestClassifier(bootstrap=False,
                              max_depth=None,
                              max_features=20,
                              max_leaf_nodes=None,
                              max_samples=None,
                              n_estimators=1000)

#Extra Random Forest

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

extraForest = ExtraTreesClassifier(n_estimators = 150)

extraForest.fit(X_train , y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=150,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
accuracy(extraForest, X_test, y_test.values)

0.7323666666666667
0.7402166666666666
0.7565



In [ ]:
S_extrab = smodel("extra150b")
S_extrab.loadModel(model = extraForest)

In [ ]:
del(extraForest)

#Ada boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time

ada = AdaBoostClassifier(
    DecisionTreeClassifier(max_leaf_nodes=200), n_estimators=500,
    algorithm="SAMME.R", learning_rate=0.5
    )

t = time.time()
ada.fit(X_train, y_train.values[:, 0])
print(time.time() - t)

1338.385749578476


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time

class adaSep(BaseEstimator, ClassifierMixin): #tienen que ser tres modelos separados
  def __init__(self, n_estimators = 1500):
    self.model1 = AdaBoostClassifier(
                                  DecisionTreeClassifier(max_leaf_nodes=200), n_estimators=n_estimators,
                                  algorithm="SAMME.R", learning_rate=0.5
                                  )
    self.model2 = AdaBoostClassifier(
                                  DecisionTreeClassifier(max_leaf_nodes=200), n_estimators=n_estimators,
                                  algorithm="SAMME.R", learning_rate=0.5
                                  )
    self.model3 = AdaBoostClassifier(
                                  DecisionTreeClassifier(max_leaf_nodes=200), n_estimators=n_estimators,
                                  algorithm="SAMME.R", learning_rate=0.5
                                  )
  def fit(self, X, y):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    y = np.asarray(y)
    self.model1.fit(X, y[:, 0])
    print("trained 1")
    self.model2.fit(X, y[:, 1])
    print("trained 2")
    self.model3.fit(X, y[:, 2])
    print("trained 3")
  def predict(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    pred_1 = self.model1.predict(X).reshape(-1, 1)
    pred_2 = self.model2.predict(X).reshape(-1, 1)
    pred_3 = self.model3.predict(X).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)
  def predict_proba(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    pred_1 = self.model1.predict_proba(X).reshape(-1, 2)
    pred_2 = self.model2.predict_proba(X).reshape(-1, 2)
    pred_3 = self.model3.predict_proba(X).reshape(-1, 2)
    return [pred_1, pred_2, pred_3]

#ada = adaSep(n_estimators = 600)

t = time.time()
#ada.fit(X_train, y_train.values)
print(time.time() - t)

3.886222839355469e-05


In [ ]:
accuracy(S_ada1, X_test, y_test.values)

In [ ]:
from sklearn.externals import joblib

joblib.dump(ada, path + "models/" + "ada200_600.pk")

['/gdrive/MyDrive/ML/DataCrunch/models/ada200_600.pk']

In [ ]:
S_ada1 = smodel("ada20_1500")
S_ada1.loadModel(model = ada)

In [ ]:
S_ada2 = smodel("ada200_600")
S_ada2.loadModel(model = ada)

#gradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gdrt = GradientBoostingRegressor(max_depth = 20, n_estimators = 30, learning_rate = 1)

gdrt.fit(X_train, y_train.values[:,0])

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1, loss='ls', max_depth=20,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=30,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
accuracy(gdrt, X_test, y_test.values)

0.9983888888888889



#Neighbourds

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors  import KNeighborsClassifier

import time

class Neigh(BaseEstimator, ClassifierMixin):
  def __init__(self, n_neighbors=3):
    self.model = KNeighborsClassifier(n_neighbors = n_neighbors, weights = "distance")
  def fit(self, X, y):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    y = np.asarray(y)
    self.model.fit(X, y)
  def predict(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    return self.model.predict(X)
  def predict_proba(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    return self.model.predict_proba(X)

In [ ]:
from sklearn.neighbors  import KNeighborsClassifier

neigh = Neigh(n_neighbors=40)
neigh.fit(X_train, y_train)

In [ ]:
accuracy(neigh, X_test, y_test.values)

0.6676666666666666
0.6762833333333333
0.6933



#Model **Class**

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.externals import joblib

"""
Para poder manejar tantos model juntos a la vez  he creado esta clase
Guarda en archivos las predicciones y los modelos
Entonces no hay que tener el modelo en ram para hacer las predicciones asi que se puden tener todos operativos
y hace las predicciones instantaneamente, ya que estan hechas de antemano

Meneja guardar y cargar los modelos de una manera bastante comoda. Puede usar un modelo, o el archivo deonde esta guardado el modelo

El plan era combinar los modelos de algina manera, pero encontre dificultades con eso
"""
class smodel(BaseEstimator, ClassifierMixin):
  def __init__(self, name, load_from_file = False):
    self.model = None
    self.y_pred_train = None
    self.y_pred_test = None
    self.y_pred_val = None
    self.name = name
    if(load_from_file):
      self.loadFile()
    pass
  def loadModel(self, model = None):
    if (model == None):
      self.model = joblib.load(path + "models/" + self.name + ".pk")
    else:
      self.model = model
      joblib.dump(self.model, path + "models/" + self.name + ".pk")

    self.y_pred_train = self.model.predict_proba(X_train)
    #if len(self.y_pred_train.shape) < 3:
    #  self.y_pred_train = np.expand_dims(self.y_pred_train, axis = -1)
      #print(self.y_pred_train.shape)
      #self.y_pred_train = np.concatenate((1- self.y_pred_train, self.y_pred_train),axis = -1)
    
    self.y_pred_test = self.model.predict_proba(X_test)
    #if len(self.y_pred_test.shape) < 3:
     # self.y_pred_test = np.expand_dims(self.y_pred_test, axis = -1)
      #self.y_pred_test = np.concatenate((1- self.y_pred_test, self.y_pred_test),axis = -1)
    
    self.y_pred_val = self.model.predict_proba(X_val)
    #if len(self.y_pred_val.shape) < 3:      
     # self.y_pred_val = np.expand_dims(self.y_pred_val, axis = -1)
      #self.y_pred_val = np.concatenate((1- self.y_pred_val, self.y_pred_val),axis = -1)
    del(self.model)
    self.save2File()
  def save2File(self):
    np.save(path + "preds/" + self.name + "_train" + ".npy", self.y_pred_train)
    np.save(path + "preds/" + self.name + "_test" + ".npy", self.y_pred_test)
    np.save(path + "preds/" + self.name + "_val" + ".npy", self.y_pred_val)
  def loadFile(self):
    self.y_pred_train = np.load(path + "preds/" + self.name + "_train" + ".npy")
    self.y_pred_test = np.load(path + "preds/" + self.name + "_test" + ".npy")
    self.y_pred_val = np.load(path + "preds/" + self.name + "_val" + ".npy")
  def fit(self, X, y):
    return self
  def predict(self, X):
    X = X.values
    self.loadFile()
   # if(self.y_pred_test == None):
    #  self.leadFile()
    if((X[0]==X_train.values[0]).all()):
      return (np.asarray(self.y_pred_train)[:, :, 1] > 0.5).astype(int).transpose()
    if((X[0]==X_test.values[0]).all()):
      return (np.asarray(self.y_pred_test)[:, :, 1] > 0.5).astype(int).transpose()
    if((X[0]==X_val.values[0]).all()):
      return (np.asarray(self.y_pred_val)[:, :, 1] > 0.5).astype(int).transpose()
  def predict_proba(self, X):
    X = X.values
    self.loadFile()
    if((X[0]==X_train.values[0]).all()):
      return self.y_pred_train
    if((X[0]==X_test.values[0]).all()):
      return self.y_pred_test
    if((X[0]==X_val.values[0]).all()):
      return self.y_pred_val


In [ ]:
S_forest = smodel("forest300", load_from_file=True)
S_forest_drop = smodel("forest_drop300", load_from_file=True)
S_forest_pca = smodel("forest_pca300", load_from_file=True)
S_forest_pcab = smodel("forest_pca300b", load_from_file=True)
S_forest_join = smodel("forest_join300", load_from_file=True)
S_extra = smodel("extra150", load_from_file=True)
S_extrab = smodel("extra150b", load_from_file=True)
S_neigh = smodel("neigh40", load_from_file=True)
S_ada1 = smodel("ada20_1500", load_from_file=True)
#S_ada2 = smodel("ada200_600", load_from_file=True)
#S_xgb = smodel("xgb20_6000", load_from_file=True)

list_models = [S_forest, S_forest_drop, S_forest_join, S_ada1]


In [ ]:
S_forest_drop = smodel("forest_drop300")
S_forest_drop.loadModel() ##cargar modelo del archivo

In [ ]:
S_neigh = smodel("neigh40")
S_neigh.loadModel(model = neigh) ##usar el modelo nigh

In [ ]:
S_ada1 = smodel("ada20_1500")
S_ada1.loadModel()

In [ ]:
accuracy(S_ada1, X_test, y_test.values)

In [ ]:
compareModel(S_forest_pca, S_xgb)

#xgboost

In [ ]:
from xgboost import XGBClassifier
import time

class XGB(BaseEstimator, ClassifierMixin):
  def __init__(self, n_estimators = 1000):
    self.model1 = XGBClassifier(max_depth=20, learning_rate=0.01, n_estimators=n_estimators, colsample_bytree=0.1)
    self.model2 = XGBClassifier(max_depth=20, learning_rate=0.01, n_estimators=n_estimators, colsample_bytree=0.1)
    self.model3 = XGBClassifier(max_depth=20, learning_rate=0.01, n_estimators=n_estimators, colsample_bytree=0.1)
  def fit(self, X, y):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    y = np.asarray(y)
    self.model1.fit(X, y[:, 0])
    print("trained 1")
    self.model2.fit(X, y[:, 1])
    print("trained 2")
    self.model3.fit(X, y[:, 2])
    print("trained 3")
  def predict(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    pred_1 = self.model1.predict(X).reshape(-1, 1)
    pred_2 = self.model2.predict(X).reshape(-1, 1)
    pred_3 = self.model3.predict(X).reshape(-1, 1)
    return np.concatenate((pred_1, pred_2, pred_3), axis = 1)
  def predict_proba(self, X):
    X = np.asarray(X)
    X = np.concatenate((X[:, 9:19], X[:, 30].reshape(-1, 1)), axis = 1)
    pred_1 = self.model1.predict_proba(X).reshape(-1, 2)
    pred_2 = self.model2.predict_proba(X).reshape(-1, 2)
    pred_3 = self.model3.predict_proba(X).reshape(-1, 2)
    return [pred_1, pred_2, pred_3]

xgb = XGB(n_estimators = 10)

t = time.time()
xgb.fit(X_train, y_train.values)
print(time.time() - t)

trained 1
trained 2
trained 3
5.315197229385376


In [ ]:
accuracy(xgb, X_test, y_test.values)

0.6253333333333333
0.6314333333333333
0.62255



In [ ]:
S_xgb = smodel("xgbborrar")
S_xgb.loadModel(model = forest)

In [ ]:
xgb.predict_proba(X_test).shape

(60000, 2, 3)

In [ ]:
S_forest_pca.predict_proba(X_test).shape

(3, 60000, 2)

In [ ]:
forest = RandomForestClassifier(n_estimators = 4)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
forest.predict_proba(X_test)[0].shape

(60000, 2)

In [ ]:
S_ada1 = smodel("ada20_1500")
S_ada1.loadModel()

In [ ]:
S_ada1.y_pred_test.shape

array([[0.5126546 , 0.515769  , 0.5167023 ],
       [0.4873454 , 0.48423102, 0.4832977 ],
       [0.50999486, 0.5100281 , 0.49830198],
       ...,
       [0.48454925, 0.4831524 , 0.48364112],
       [0.49811202, 0.49936587, 0.51253474],
       [0.501888  , 0.50063413, 0.48746523]], dtype=float32)

#Compare all the models


In [ ]:
from scipy import stats

stats.spearmanr(S_extra.predict(X_test)[:, 0], S_extrab.predict(X_test)[:, 0])[0]

0.8573059117009949

In [ ]:
from scipy import stats

for model1 in list_models:
  for model2 in list_models:
    stat = 0
    for i in range(3):
      stat += stats.spearmanr(model1.predict(X_test)[:, 0], model2.predict(X_test)[:, 0])[0]
    print(stat/3 , end = "\t")
  print()

1.0	0.8225459286698866	0.553902950765111	0.736014262486994	0.5511828798038776	0.5865488020438108	0.3911569732077113	
0.8225459286698866	1.0	0.5754493752992056	0.7776381830556712	0.531995044184726	0.579968837668071	0.4069015390661946	
0.553902950765111	0.5754493752992056	1.0	0.6730717107911391	0.5385969007453659	0.42693765170837983	0.529571880797586	
0.7360142624869942	0.7776381830556712	0.6730717107911391	0.9999999999999999	0.5414962914316352	0.5363905458801111	0.45021938803433875	
0.5511828798038777	0.531995044184726	0.5385969007453659	0.5414962914316354	1.0	0.4172853330309541	0.4403166924598725	
0.5865488020438107	0.5799688376680711	0.42693765170837983	0.5363905458801111	0.4172853330309541	1.0	0.32368952614554874	
0.39115697320771137	0.4069015390661946	0.529571880797586	0.4502193880343388	0.44031669245987254	0.32368952614554874	1.0	


In [ ]:
for model in list_models:
  print(model.name)
  accuracy(model, X_test, y_test.values)
  accuracy(model, X_val, y_val.values)

forest300
0.8382
0.8458166666666667
0.8534333333333334

0.8408833333333333
0.8457333333333333
0.8555333333333334

forest_drop300
0.8228666666666666
0.83105
0.8415

0.8264333333333334
0.8319166666666666
0.8410666666666666

forest_pca300
0.7349666666666667
0.7471333333333333
0.7625

0.7382833333333333
0.7437166666666667
0.7638833333333334

forest_join300
0.7996666666666666
0.8078666666666666
0.81885

0.8017833333333333
0.80765
0.8180333333333333

extra150
0.7323666666666667
0.7402166666666666
0.7565

0.7293
0.7389666666666667
0.7588333333333334

ada20_1500
0.7889333333333334
0.7936833333333333
0.8078333333333333

0.7896333333333333
0.7974833333333333
0.8086333333333333

neigh40
0.6676666666666666
0.6762833333333333
0.6933

0.6677166666666666
0.67635
0.69175



In [ ]:
a = np.zeros_like(y_test.values).astype(float)
for model in list_models:
  a += np.asarray(model.predict(X_test))
  print(model.name)
a = a/len(list_models)
a = (a > 0.5).astype(int)

forest300
forest_drop300
forest_join300
ada20_1500


In [ ]:
  compare([a[:, 0], y_test.values[:, 0]], y_test.values[:, 0].reshape(-1, 1))
  compare([a[:, 1], y_test.values[:, 1]], y_test.values[:, 1].reshape(-1, 1))
  compare([a[:, 2], y_test.values[:, 2]], y_test.values[:, 2].reshape(-1, 1))

[0. 0. 0.] 	 0.50915
[0. 0. 1.] 	 0.0
[0. 1. 0.] 	 0.0
[0. 1. 1.] 	 0.10398333333333333
[1. 0. 0.] 	 0.0664
[1. 0. 1.] 	 0.0
[1. 1. 0.] 	 0.0
[1. 1. 1.] 	 0.3204666666666667

[0. 0. 0.] 	 0.5197
[0. 0. 1.] 	 0.0
[0. 1. 0.] 	 0.0
[0. 1. 1.] 	 0.10138333333333334
[1. 0. 0.] 	 0.06208333333333333
[1. 0. 1.] 	 0.0
[1. 1. 0.] 	 0.0
[1. 1. 1.] 	 0.31683333333333336

[0. 0. 0.] 	 0.5479833333333334
[0. 0. 1.] 	 0.0
[0. 1. 0.] 	 0.0
[0. 1. 1.] 	 0.10193333333333333
[1. 0. 0.] 	 0.05366666666666667
[1. 0. 1.] 	 0.0
[1. 1. 0.] 	 0.0
[1. 1. 1.] 	 0.29641666666666666



In [ ]:
np.asarray(S_forest.predict_proba(X_test)).shape

(3, 60000, 2)

#Combine models: Model voting

No funciono, habria que combinarlos de otra manera, como usar las predicciones como features y entrenar un modelo que las use para relizar las predicciones

In [ ]:
from sklearn.ensemble import VotingRegressor

voting_clf = VotingClassifier(
    estimators=[(model.name, model) for model in list_models],
    voting = "soft"
)
voting_clf.fit(X_test, y_test.values[:, 0])

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


VotingClassifier(estimators=[('forest300',
                              smodel(load_from_file=None, name='forest300')),
                             ('forest_drop300',
                              smodel(load_from_file=None,
                                     name='forest_drop300')),
                             ('forest_join300',
                              smodel(load_from_file=None,
                                     name='forest_join300')),
                             ('ada20_1500',
                              smodel(load_from_file=None, name='ada20_1500'))],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [ ]:
voting_clf.predict(X_test)

array([0, 1, 1, ..., 1, 0, 1])

In [ ]:
accuracy(voting_clf, X_val, y_val.values)

0.8339



In [ ]:
for model in list_models:
  print(model.name, model.y_pred_test[0])

forest300 [[1.         0.        ]
 [0.47666667 0.52333333]
 [0.01333333 0.98666667]
 ...
 [0.15333333 0.84666667]
 [1.         0.        ]
 [0.05       0.95      ]]
forest_drop300 [[1.         0.        ]
 [0.52666667 0.47333333]
 [0.09       0.91      ]
 ...
 [0.12       0.88      ]
 [1.         0.        ]
 [0.05333333 0.94666667]]
forest_pca300 [[0.99333333 0.00666667]
 [0.48       0.52      ]
 [0.05666667 0.94333333]
 ...
 [0.39333333 0.60666667]
 [0.77333333 0.22666667]
 [0.37666667 0.62333333]]
forest_join300 [[1.         0.        ]
 [0.41666667 0.58333333]
 [0.03333333 0.96666667]
 ...
 [0.29333333 0.70666667]
 [1.         0.        ]
 [0.16333333 0.83666667]]
extra150 [[0.92666667 0.07333333]
 [0.3        0.7       ]
 [0.08666667 0.91333333]
 ...
 [0.36       0.64      ]
 [0.83333333 0.16666667]
 [0.26666667 0.73333333]]
neigh40 [[0.94649888 0.05350112]
 [0.44606023 0.55393977]
 [0.43672286 0.56327714]
 ...
 [0.48703731 0.51296269]
 [0.90675317 0.09324683]
 [0.42990104 0.5700

In [ ]:
X_preds = np.zeros_like(y_test.values[:, 0])
for model in list_models:
  X_preds = np.concatenate(a, model.predict(X_test), axis = 1)


NameError: ignored

In [ ]:
final = 

In [ ]:
X_preds.shape

NameError: ignored

#Predict


In [ ]:
hackathon_data_clean = clean_data(hackathon_data, testing = True)
hackathon_data_clean.describe()

In [ ]:
prediction = pd.DataFrame()

prediction['target_r'] = y_pred_h[:,0]
prediction['target_g'] = y_pred_h[:,1]
prediction['target_b'] = y_pred_h[:,2]

In [ ]:
prediction.describe()

#Submit predictions


In [ ]:
API_KEY = "____"

r = requests.post("https://hackathon.datacrunch.com/api/submit",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("The submissions are not enabled because the hackathon hasn't started yet or is already finished.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same a the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")